In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import cv2
import os
import json
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
mpl.rcParams['figure.figsize'] = (6, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
# df = pd.read_csv('cleaned_imgs_df.csv')
df = pd.read_csv('imputed_imgs_df.csv')
df1 = pd.read_csv('cleaned_imgs_df.csv')

### Test Data Analysis

In [ ]:
# Plotting correlation between every pair of columns
# plt.rcParams['figure.figsize'] = [15, 15]
# sns.heatmap(imputed_df.corr(),annot = True,cmap = "YlGnBu")
# plt.title('Correlation Matrix')
# plt.show()

In [ ]:
# start_time.describe()

In [ ]:
# mpl.rcParams['figure.figsize'] = (6, 6)
# start_time.hist()
# # Start time is from 4 30 am to 7 30 am

In [ ]:
# end_time.describe()

In [ ]:
# end_time.hist()
# # end time is 8am - 5pm

In [ ]:
# df_list[220]

In [ ]:
# (df_list[220][df_list[0].columns[2]]).plot()

In [ ]:
# length.describe()

## Split data and Normalize

In [4]:
cleaned_df = pd.read_csv('cleaned_df.csv')
imputed_df = pd.read_csv('imputed_df.csv')

In [5]:
from datetime import date
def calc_time(time):
    return int(time.split(':')[0])*60 + int(time.split(':')[1])
def calc_day(day):
    x = date(int(day.split('/')[2]), int(day.split('/')[0]),int(day.split('/')[1]))
    if(day.split('/')[2]=='2017'):
        return x.timetuple().tm_yday-365+1
    if(day.split('/')[2]=='2018'):
        return x.timetuple().tm_yday
    if(day.split('/')[2]=='2019'):
        return x.timetuple().tm_yday+365
    if(day.split('/')[2]=='2020'):
        return x.timetuple().tm_yday+365*2
    if(day.split('/')[2]=='2021'):
        return x.timetuple().tm_yday+365*3+1
def test_calc_day(day):
    x = date(int(day.split('/')[0]), int(day.split('/')[1]),int(day.split('/')[2]))
    if(day.split('/')[0]=='2017'):
        return x.timetuple().tm_yday-365+1
    if(day.split('/')[0]=='2018'):
        return x.timetuple().tm_yday
    if(day.split('/')[0]=='2019'):
        return x.timetuple().tm_yday+365
    if(day.split('/')[0]=='2020'):
        return x.timetuple().tm_yday+365*2
    if(day.split('/')[0]=='2021'):
        return x.timetuple().tm_yday+365*3+1

In [6]:
df_list = [None]*300
for i in range(1,301):
    t_df = pd.read_csv('data_to_nvidia/Data_Test/{}/weather_data.csv'.format(i))
    t_df['Time'] = [calc_time(j) for j in t_df['MST']]
    t_df['day'] = [test_calc_day(j) for j in t_df['DATE (YYYY/MM/DD)']]
    df_list[i-1] = t_df

In [7]:
start_time = []
end_time = []
length = []
test_days_list = []
for t_df in df_list:
    start_time.append(t_df['Time'][0])
    length.append(len(t_df))
    end_time.append(t_df['Time'][len(t_df)-1])
    test_days_list.append(t_df['day'][0])
start_time = pd.Series(start_time)
end_time = pd.Series(end_time)
length = pd.Series(length)

test_days_list_unique = list(pd.Series(test_days_list).unique())

In [8]:
# def fetch(tup):
#     day,start_time,end_time = tup
#     df = imputed_df[imputed_df['day']==day]
#     return df.loc[(df['Time'] >= start_time) & (df['Time'] <= end_time)].reset_index(drop=True)

In [9]:
# Split into Train and Validation data

X = df['day'].unique()
from sklearn.utils import shuffle
X = shuffle(X, random_state = 42)
X = list(X)
for i in test_days_list_unique:
    X.remove(i)
    
train_days = X[0:int(len(X)*0.85)]
val_days = X[int(len(X)*0.85):]
test_days = test_days_list_unique

def get_dataframe(dataf,days_list):
  indexes_list = []
  for day in days_list:
    indexes_list = np.append(indexes_list, dataf[dataf['day'] == day].index)
  return dataf.iloc[np.array(indexes_list).astype('int64')]

In [ ]:
# train_df

In [10]:
cols = ['day', 'Normal imgs NE', 'Normal imgs UE','Proj imgs NE', 'Proj imgs UE', 'BRBG imgs', 'CDOC imgs']
train_df = get_dataframe(df, train_days).drop(['MST','Date','MST_img', 'Date_img'], axis = 1)
val_df = get_dataframe(df, val_days).drop(['MST','Date','MST_img', 'Date_img'], axis = 1)
test_df = get_dataframe(df, test_days).drop(['MST','Date','MST_img', 'Date_img'], axis = 1)

a = train_df[cols]
b = val_df[cols]
c = test_df[cols]

In [13]:
# Normalize the data
train_mean = train_df.mean(numeric_only = True)
train_std = train_df.std(numeric_only = True)

train_df = (train_df- train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std
train_df[cols] = a
val_df[cols] = b
test_df[cols] = c

In [ ]:
# full_test_dfs = [fetch(i).drop(['MST','Date'], axis = 1) for i in list(zip(test_days_list,start_time,end_time))]

# for i in range(len(full_test_dfs)):
#        full_test_dfs[i] = (full_test_dfs[i]- train_mean) / train_std

In [11]:
def create_submission(x,path):
  x.index = list(range(1,301))
  x.to_csv(path,header=['10_min_horizon','20_min_horizon','30_min_horizon','40_min_horizon','50_min_horizon', '60_min_horizon','70_min_horizon','80_min_horizon','90_min_horizon','100_min_horizon','110_min_horizon','120_min_horizon'], index=True, index_label = 'scenario_set')

In [ ]:
#Ensembling

# df_nn = pd.read_csv('/content/sub.csv')
# df_loocf = pd.read_csv('/content/sub3.csv')
# x = (df_nn + df_loocf) / 2
# create_submission(x.drop('scenario_set', axis = 1),'/content/sub1.csv' )

In [12]:
def get_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_LINEAR)
    return img

def get_images_list(paths):
    images_list = []
    for path in paths:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_LINEAR)
        images_list.append(img)
    return np.array(images_list)

In [ ]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [ ]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [ ]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [ ]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

In [ ]:
# mpl.rcParams['figure.figsize'] = (12, 12)
# f, axarr = plt.subplots(1,2)
# axarr[0].imshow(img)
# axarr[1].imshow(img1)

### Using Deep Learning Models

In [ ]:
# # extract test data
# import os
# test_data = []*300
# for i in range(1,301):
#   t_df = pd.read_csv(os.path.join('/content/drive/MyDrive/Shell_ai/dataset/test', str(i), "weather_data.csv")) 
#   t_df['Total Cloud Cover [%]'].replace(-7999, np.NaN, inplace  = True)
#   t_df['Total Cloud Cover [%]'].replace(-1, np.NaN, inplace  = True)
#   t_df['Total Cloud Cover [%]'].replace(-6999, np.NaN, inplace  = True) 
#   t_df['Total Cloud Cover [%]'].fillna(method="ffill", inplace=True)
#   t_df['Total Cloud Cover [%]'].fillna(method='backfill', inplace=True)
#   t_df.drop('Time [Mins]', axis = 1, inplace = True)
#   wv = t_df.pop('Peak Wind Speed @ 6ft [m/s]')
#   # Convert to radians.
#   wd_rad = t_df.pop('Avg Wind Direction @ 6ft [deg from N]')*np.pi / 180
#   # Calculate the wind x and y components.
#   t_df['Wind_x'] = wv*np.cos(wd_rad)
#   t_df['Wind_y'] = wv*np.sin(wd_rad)
#   t_df = (t_df- train_mean.drop('day')) / train_std.drop('day')
#   test_data.append(t_df)

In [14]:
# tf.random.set_seed(42)
# np.random.seed(42)

non_image_cols1 = ['Albedo',
 'Azimuth Angle',
 'Clear Sky GHI',
 'DNI',
 'Declination Angle',
 'Dew Temp',
 'Dry Temp',
 'GHI',
 'Moisture',
 'Precipitation',
 'Pressure',
 'RH',
 'Snow Depth',
 'TCC',
 'Time',
 'Wet Temp',
 'Wind_x',
 'Wind_y',
 'Zenith Angle',
 'day',
 'distance factor']

non_image_cols = ['Albedo',
 'Azimuth Angle',
 'Clear Sky GHI',
 'DNI',
 'Declination Angle',
 'Dew Temp',
 'Dry Temp',
 'GHI',
 'Moisture',
 'Precipitation',
 'Pressure',
 'RH',
 'Snow Depth',
 'TCC',
 'Wet Temp',
 'Wind_x',
 'Wind_y',
 'day',
 'distance factor']

In [ ]:
a_file = open("all_images.json", "r")
Proj_imgs_NE = a_file.read()
a_file.close()

In [ ]:
json_load =  json.loads(Proj_imgs_NE)

In [ ]:
# for path in df['Normal_imgs_NE']

In [15]:
def get_data(ws, dataframe, image_type = ['Proj imgs NE'],dataframe_cols_list = non_image_cols):
    df = dataframe.copy(deep = False)
    df.reset_index(drop = True, inplace = True)
    imgs_df = df[image_type[0]]
    df = df[dataframe_cols_list]
    image_array = np.array([json_load[path] for path in imgs_df ])
    data = []
    images = []
    targets = []
    grouped_df = df.groupby(['day'], sort = False, as_index = False)
    total_window_size = ws + 12
    for day in df['day'].unique():
        day_size = grouped_df.get_group(day)['GHI'].count()
        offset = grouped_df.get_group(day).index.values[0]
        day_df = grouped_df.get_group(day).reset_index(drop = True)
        day_df.drop('day',axis = 1,inplace = True)
        if(day_size<total_window_size):
            pass
        else:
            for i in range(day_size - total_window_size + 1):
                targets.append(np.array(day_df['GHI'].iloc[list(range(ws+i,ws+i+12))]))
                images.append(image_array[offset+i:offset+ws+i])
                data.append(np.array((day_df.iloc[list(range(i,ws+i))])))
    return np.array(data),np.array(images),np.array(targets)

In [ ]:
# X_train,imgs_train,y_train = get_data(12,train_df)

In [ ]:
# X_val,imgs_val,y_val = get_data(12,val_df)

In [ ]:
# X_test,imgs_test,y_test = get_data(12,test_df)

In [ ]:
# np.save('X_train', X_train)
# np.save('imgs_train', imgs_train)
# np.save('y_train', y_train)

# np.save('X_val', X_val)
# np.save('imgs_val', imgs_val)
# np.save('y_val', y_val)

# np.save('X_test', X_test)
# np.save('imgs_test', imgs_test)
# np.save('y_test', y_test)

In [16]:
# loaded_array = np.load('file_name.npy')

X_train = np.load('X_train.npy')
imgs_train = np.load('imgs_train.npy')
y_train = np.load('y_train.npy' )

X_val = np.load('X_val.npy')
imgs_val = np.load('imgs_val.npy')
y_val = np.load('y_val.npy' )

X_test = np.load('X_test.npy')
imgs_test = np.load('imgs_test.npy')
y_test = np.load('y_test.npy' )

In [17]:
def DATALOADER(data, imgs, y, bs):
    i = 0
    length = len(y)
    while True:
        batch_data = []
        batch_imgs = []
        batch_labels = []
        while len(batch_data) < bs:
              batch_data.append(data[i])
              batch_imgs.append(imgs[i])
              batch_labels.append(y[i])
              i += 1
              if i >= length:                    # Reset if reached end of paths
                  i = 0
                  shuffler = np.random.permutation(len(y))
                  data = data[shuffler]
                  y = y[shuffler]
                  imgs = imgs[shuffler]
        # You can call one method inside another method using self.method_name(arugments)
        # Note that you don't have to pass "self" while calling the method, just the remaining arguments  

        # yield the batch to the calling function
        yield ({"input_x": np.array(batch_data), "input_y": np.array(batch_imgs)}, np.array(batch_labels))

In [ ]:
def DATALOADER(imgs, y, bs):
    i = 0
    length = len(y)
    while True:
#         batch_data = []
        batch_imgs = []
        batch_labels = []
        while len(batch_labels) < bs:
#               batch_data.append(data[i])
              batch_imgs.append(imgs[i])
              batch_labels.append(y[i])
              i += 1
              if i >= length:                    # Reset if reached end of paths
                  i = 0
                  shuffler = np.random.permutation(len(y))
#                   data = data[shuffler]
                  y = y[shuffler]
                  imgs = imgs[shuffler]
        # You can call one method inside another method using self.method_name(arugments)
        # Note that you don't have to pass "self" while calling the method, just the remaining arguments  

        # yield the batch to the calling function
        yield ({"input_cnn":np.array(batch_imgs)}, np.array(batch_labels))

In [ ]:
# # mean_pixel = [92.39862040529066, 126.61433527436093, 150.58130643035432]
# mean_pixel = []
# for c in range(3):
#     mean_pixel.append(imgs_train[:,:,:,:,c].mean())

In [ ]:
#  for c in range(3):
#     imgs_train[:,:,:,:,c] = imgs_train[:,:,:,:,c] - mean_pixel[c]
#     imgs_val[:,:,:,:,c] = imgs_val[:,:,:,:,c] - mean_pixel[c]
#     imgs_test[:,:,:,:,c] = imgs_test[:,:,:,:,c] - mean_pixel[c]

In [ ]:
# for c in range(3):
#     print("\n","c =",c )
#     print(imgs_train[:,:,:,:,c].mean())
#     print(imgs_val[:,:,:,:,c].mean())
#     print(imgs_test[:,:,:,:,c].mean())

In [ ]:
# plt.imshow(imgs_train[9,4,:,:,2])

In [ ]:
imgs_train = imgs_train/162
imgs_val = imgs_val/162
imgs_test = imgs_test/162

In [ ]:
imgs_train.max()*255

In [ ]:
# day_df = grouped_df.get_group(13).reset_index(drop = True)
# day_df.drop(['day','BRBG imgs','CDOC imgs','Normal imgs NE','Normal imgs UE','Proj imgs UE'],axis = 1,inplace = True)
# day_df

### CNN extract features

In [ ]:
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.vgg16 import preprocess_input

# vgg16_input = tf.keras.layers.Input(shape=(64,64,3),name = "input_cnn")

# ### CNN ###
# # x = preprocess_input(vgg16_input)
# vgg = VGG16(weights=None, include_top=False, input_shape = (64,64,3))
  
# # x = preprocess_input(vgg16_input)
# x = vgg(vgg16_input)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.4)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# # x = tf.keras.layers.Dense(256, activation='relu')(x)
# # x = tf.keras.layers.Dropout(0.2)(x)

# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.1)(x)
# # x = tf.keras.layers.Dropout(0.1)(x)


# x = tf.keras.layers.Dense(1, activation='linear')(x)

# cnn_model = tf.keras.Model(inputs=vgg16_input, outputs=x)

In [91]:
layer = tf.keras.layers
cnn_input = tf.keras.layers.Input(shape=(64,64,3),name = "input_cnn")

cnn_output = tf.keras.layers.Conv2D(128,7,padding="same", dilation_rate=(4, 4), activation='leaky_relu')(cnn_input)

cnn_output = tf.keras.layers.Conv2D(64,3,padding="same", activation='leaky_relu')(cnn_output)
cnn_output = tf.keras.layers.MaxPooling2D((2, 2))(cnn_output)

cnn_output = tf.keras.layers.Conv2D(128,3,padding="same", activation='leaky_relu')(cnn_output)
cnn_output = tf.keras.layers.Conv2D(128,3,padding="same", activation='leaky_relu')(cnn_output)
cnn_output = tf.keras.layers.MaxPooling2D((2, 2))(cnn_output)

cnn_output = tf.keras.layers.Conv2D(256,3,padding="same", activation='leaky_relu')(cnn_output)
cnn_output = tf.keras.layers.Conv2D(256,3,padding="same", activation='leaky_relu')(cnn_output)
cnn_output = tf.keras.layers.Conv2D(256,3,padding="same", activation='leaky_relu')(cnn_output)
cnn_output = tf.keras.layers.MaxPooling2D((2, 2))(cnn_output)
cnn_output = layer.Flatten()(cnn_output)
cnn_output = layer.Dropout(0.2)(cnn_output)
cnn_output = layer.Dense(512,activation='relu')(cnn_output)
x = layer.Dropout(0.3)(cnn_output)

# cnn_output = layer.Dense(128,activation='leaky_relu')(cnn_output)
# cnn_output = layer.Dropout(0.2)(cnn_output)

x = layer.Dense(64, activation='leaky_relu')(x)
x = layer.Dropout(0.2)(x)

x = layer.Dense(1)(x)

cnn_model_atrous = tf.keras.Model(inputs=cnn_input, outputs=x)

In [92]:
cnn_model_atrous.compile(loss=tf.keras.losses.Huber(),
                optimizer=tf.optimizers.Adam(learning_rate=1e-3 ,decay=1e-3 /1000),
                metrics=[tf.metrics.MeanAbsoluteError()])
cnn_model_atrous.load_weights('vgg16_final209.h5')

In [94]:
feature_extractor = tf.keras.Model(inputs=cnn_model_atrous.input, outputs=[x])
feature_extractor.trainable = False

In [ ]:
# add_model = tf.keras.Sequential()
# # add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
# add_model.add(layer.Dense(256, activation='relu'))
# add_model.add(layer.Dense(32, activation='relu'))
# add_model.add(layer.Dense(1))

# tune_model = tf.keras.Model(inputs=cnn_model_atrous.input, outputs=add_model(cnn_model_atrous.output))

# # tune_model.summary()

In [ ]:
X_images = np.array([json_load[path] for path in df1['Proj imgs NE'] ])

In [ ]:
y =  np.array(df1['GHI'])

In [ ]:
# X_images = np.load('images_for_Feature Extraction.npy')
# y = np.load('images_for_Feature Extraction_y.npy')

In [ ]:
X_images.dtype

In [ ]:
n = len(y)
X_images_train = X_images[:int(0.9*n)]
y_train = y[:int(0.9*n)]

X_images_val = X_images[int(0.9*n): ]
y_val = y[int(0.9*n):]

# X_images_test = X_images[int(0.9*n):]
# y_test = y[int(0.9*n):]

In [ ]:
mean_pixel = []
for c in range(3):
    mean_pixel.append(X_images_train[:,:,:,c].mean())

In [ ]:
 for c in range(3):
    X_images_train[:,:,:,c] = X_images_train[:,:,:,c] - mean_pixel[c]
    X_images_val[:,:,:,c] = X_images_val[:,:,:,c] - mean_pixel[c]
#     X_images_test[:,:,:,c] = X_images_test[:,:,:,c] - mean_pixel[c]

In [ ]:
X_images_train = X_images_train/(X_images_train.max())
X_images_val = X_images_val/(X_images_val.max())
# X_images_train = X_images_train/(X_images_train.max())

In [ ]:
bs = 100
images_train_gen = DATALOADER(X_images_train, y_train, bs)
images_val_gen = DATALOADER(X_images_val, y_val, 16)
# images_test_gen = DATALOADER(X_images_test, y_test, 32)

train_steps = len(y_train)/bs
val_steps = len(y_val)/16
# test_steps = len(y_test)/32

In [ ]:
i = i + 1
callbacks = [
#       EarlyStopping(monitor='val_loss', patience=9,  mode='min',verbose=1),
      ReduceLROnPlateau(factor=0.2, patience = 3, min_lr=1e-12, verbose=1),
      ModelCheckpoint('vgg16_final{}.h5'.format(i), verbose=1, save_best_only=True, save_weights_only=True) ]

# lr_schedule = tf.keras.callbacks.LearningRateScheduler(
#     lambda epoch: 1e-8 * 10**(epoch / 10))

cnn_model_atrous.compile(loss=tf.keras.losses.Huber(),
                optimizer=tf.optimizers.Adam(learning_rate=1e-3 ,decay=1e-3 /1000),
                metrics=[tf.metrics.MeanAbsoluteError()])

In [ ]:
history = cnn_model_atrous.fit(images_train_gen, steps_per_epoch = train_steps,epochs = 200,validation_data = images_val_gen,validation_steps =  val_steps,callbacks = [callbacks])
# model.evaluate(images_test_gen)

### MLP

In [ ]:
# ###from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input

# vgg16_input = tf.keras.layers.Input(shape=(64,64,3))

# ### CNN ###
# vgg = VGG16(input_tensor=vgg16_input, weights='imagenet', include_top=False)

# x = preprocess_input(vgg16_input)

# x = vgg(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.4)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)

# cnn_model = tf.keras.Model(inputs=vgg16_input, outputs=x)

# vgg16_input = tf.keras.layers.Input(shape=(64,64,3))

# ### CNN ###
# vgg = VGG16(input_tensor=vgg16_input, weights='imagenet', include_top=False)

# x = vgg(vgg16_input)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.4)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)

# cnn_model = tf.keras.Model(inputs=vgg16_input, outputs=x)

In [ ]:
# # input1 = tf.keras.layers.Input(shape=(12,len(non_image_cols)-1 ), name='input_x')
# # Only for images #
# input2 = tf.keras.layers.Input(shape=(12,64,64,3),  name='input_y')

# # y = aux_model(input1)

# x = tf.keras.layers.TimeDistributed(cnn_model, input_shape=(12,64,64,3))(input2)
# x = tf.keras.layers.LSTM(128,return_sequences = True)(x)
# # x = tf.keras.layers.LSTM(16,return_sequences = True)(t)

# # combinedInput = tf.keras.layers.Concatenate()([x, y])
# # f = tf.keras.layers.LSTM(128,return_sequences = True)(combinedInput)
# f = tf.keras.layers.LSTM(128)(x)
# f = tf.keras.layers.Dropout(0.5)(f)
# # f = tf.keras.layers.Dense(256, activation="relu")(f)
# # f = tf.keras.layers.Dropout(0.5)(f)
# f = tf.keras.layers.Dense(64, activation="relu")(f)
# f = tf.keras.layers.Dropout(0.2)(f)
# f = tf.keras.layers.Dense(12, activation="linear")(f)
# images_only_model = tf.keras.Model(inputs=input2, outputs=f)

In [104]:
aux_input = tf.keras.layers.Input(shape=(12,len(non_image_cols)-1 ))

## MLP ###
t = tf.keras.layers.Flatten()(mlp_input)
t = tf.keras.layers.Dense(64)(aux_input)
t = tf.keras.layers.Dropout(0.3)(t)
t = tf.keras.layers.Dense(16,activation = 'leaky_relu')(t)
# t = tf.keras.layers.Dense(units=12,activation='leaky_relu')(t)

aux_model = tf.keras.Model(inputs=aux_input, outputs=t)

TypeError: ('Keyword argument not understood:', 'return_sequences')

In [85]:
aux_model.output

<KerasTensor: shape=(None, 12, 8) dtype=float32 (created by layer 'dense_28')>

In [ ]:
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.vgg16 import preprocess_input

# vgg16_input = tf.keras.layers.Input(shape=(64,64,3))

# ### CNN ###
# # x = preprocess_input(vgg16_input)
# vgg = VGG16(weights=None, include_top=False, input_shape = (64,64,3))
# x = vgg(vgg16_input)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.4)(x)
# x = tf.keras.layers.Dense(256, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)

# cnn_model = tf.keras.Model(inputs=vgg16_input, outputs=x)

In [101]:
input1 = tf.keras.layers.Input(shape=(12,len(non_image_cols)-1 ), name='input_x')
input2 = tf.keras.layers.Input(shape=(12,64,64,3),  name='input_y')

y = aux_model(input1)

x = tf.keras.layers.TimeDistributed(feature_extractor, input_shape=(12,64,64,3))(input2)
x = tf.keras.layers.LSTM(128,return_sequences = True, recurrent_dropout = 0.4)(x)
# x = tf.keras.layers.LSTM(16,return_sequences = True)(t)

combinedInput = tf.keras.layers.Concatenate()([x, y])
f = tf.keras.layers.Dropout(0.3)(f)
f = tf.keras.layers.LSTM(64,recurrent_dropout = 0.2)(combinedInput)
f = tf.keras.layers.Dropout(0.2)(f)
# f = tf.keras.layers.Dense(256, activation="relu")(f)
# f = tf.keras.layers.Dropout(0.5)(f)
f = tf.keras.layers.Dense(32, activation="leaky_relu")(f)
f = tf.keras.layers.Dropout(0.1)(f)
f = tf.keras.layers.Dense(12, activation="linear")(f)
mixed_model = tf.keras.Model(inputs=[input1,input2], outputs=f)

In [99]:
mixed_model.summary()

Model: "model_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_y (InputLayer)            [(None, 12, 64, 64,  0                                            
__________________________________________________________________________________________________
time_distributed_11 (TimeDistri (None, 12, 1)        10211521    input_y[0][0]                    
__________________________________________________________________________________________________
input_x (InputLayer)            [(None, 12, 18)]     0                                            
__________________________________________________________________________________________________
lstm_39 (LSTM)                  (None, 12, 128)      66560       time_distributed_11[0][0]        
___________________________________________________________________________________________

In [25]:
imgs_train = imgs_train/(imgs_train).max()
imgs_val = imgs_val/(imgs_train).max()
imgs_test = imgs_test/(imgs_train).max()

In [88]:
bs = 80

train_gen = DATALOADER(X_train,imgs_train, y_train, bs)
train_steps = len(X_train)/bs

val_gen = DATALOADER(X_val,imgs_val, y_val, 8)
val_steps = len(X_val)/8

test_gen = DATALOADER(X_test,imgs_test, y_test, 32)
test_steps = len(X_test)/32

In [57]:
x,y = iter(next(train_gen))

In [102]:
i = i + 1
callbacks = [
      EarlyStopping(monitor='val_loss', patience=50,  mode='min',verbose=1),
      ReduceLROnPlateau(factor=0.1, patience = 3, min_lr=1e-10, verbose=1),
      ModelCheckpoint('mixed_model_final{}.h5'.format(i), verbose=1, save_best_only=True, save_weights_only=True) ]

mixed_model.compile(loss=tf.keras.losses.Huber(),
                optimizer=tf.optimizers.Adam(learning_rate=1e-3 ,decay=1e-3 /50),
                metrics=[tf.metrics.MeanAbsoluteError()])

In [103]:
history = mixed_model.fit(train_gen, steps_per_epoch = train_steps,validation_data = val_gen,validation_steps = val_steps,epochs = 100,callbacks = [callbacks])
# model.evaluate(test_gen)

Epoch 1/100
504/504 [==============================] - 183s 353ms/step - loss: 0.1873 - mean_absolute_error: 0.4602 - val_loss: 0.4204 - val_mean_absolute_error: 0.7917

Epoch 00001: val_loss improved from inf to 0.42037, saving model to mixed_model_final214.h5
Epoch 2/100
504/504 [==============================] - 177s 352ms/step - loss: 0.1491 - mean_absolute_error: 0.3918 - val_loss: 0.3342 - val_mean_absolute_error: 0.6763

Epoch 00002: val_loss improved from 0.42037 to 0.33418, saving model to mixed_model_final214.h5
Epoch 3/100
116/504 [=====>........................] - ETA: 1:43 - loss: 0.1495 - mean_absolute_error: 0.3873

KeyboardInterrupt: 

In [ ]:
mixed_model.summary()

In [ ]:
callbacks = [
      EarlyStopping(monitor='val_loss', patience=9,  mode='min',verbose=1),
      ReduceLROnPlateau(factor=0.15, patience=5, min_lr=1e-8, verbose=1),
      ModelCheckpoint('Mixed_model.h5', verbose=1, save_best_only=True, save_weights_only=True) ]

model.compile(loss=tf.keras.losses.Huber(),
                optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                metrics=[tf.metrics.MeanAbsoluteError()])

history = model.fit(x=[X_train, imgs_train], y=y_train,
     validation_data=([X_val, imgs_val], y_val),max_queue_size = 2,
     epochs=20, batch_size=128 callbacks=callbacks)

# model.evaluate(test_gen)

In [ ]:
def get_test_predictions(model) :

  X = np.zeros((300,ws,len(features_list)))
  for i in range(300):  
    X[i] = np.array(full_test_dfs[i][-12:])
  output = model.predict(X)
  output = output.reshape(300,12)
  output = pd.DataFrame(output)
#     print(output)
  output = output*train_std.loc['GHI'] + train_mean.loc['GHI']
  for i in range(300):
    for j in range(4):
      if(output.iloc[i][j] < -1):
          output.iloc[i][j] = -0.8
  create_submission(pd.DataFrame(np.array(output)),'predictions_final_1.csv')
  return output

In [ ]:
model.built = True
model.load_weights('MLP_l:0.01_sr:10_ws:12_u1:256_u2:32_cols:2.h5')
model.evaluate(test_gen, steps = test_batches)

In [ ]:
get_test_predictions(model,12,l[2],10)

In [ ]:
mpl.rcParams['figure.figsize'] = (8,8)
day = 100
grouped_imputed_df = imputed_df.groupby(['day'], sort = False, as_index = False)
def plot(i):
    pd.Series(x.iloc[i].values , index=range(len(full_test_dfs[i])+10,len(full_test_dfs[i])+121,10)).plot()
    (grouped_imputed_df.get_group(test_days_list[i])['GHI'].reset_index(drop = True)).plot()

### CNN

In [ ]:
def Model(filters,kernel_size,ws,i,units_1):

    model =   tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=filters,kernel_size=kernel_size,activation='relu'
                               ,input_shape=(ws,len(l[i]))),
        tf.keras.layers.MaxPooling1D(pool_size = 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=units_1, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(units=12),
    ])
    model.summary()
    return model

# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation= ' relu ' ,
# input_shape=(n_timesteps,n_features)))
# model.add(Conv1D(filters=32, kernel_size=3, activation= ' relu ' ))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=16, kernel_size=3, activation= ' relu ' ))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(100, activation= ' relu ' ))
model.add(Dense(n_outputs))